In [1]:
from datasets import load_dataset 

# load a custom dataset from local/remote files or folders using the ImageFolder feature

dataset = load_dataset("imagefolder", data_dir="/kaggle/input/thermography-images-of-diabetic-foot/ThermoDataBase/val", split="train")
dataset

Resolving data files:   0%|          | 0/422 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 422
})

In [2]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[1]

'DM Group'

In [3]:
import glob

paths = glob.glob("/kaggle/input/thermography-images-of-diabetic-foot/ThermoDataBase/val/*/*")
len(paths)

422

In [4]:
import numpy as np

In [5]:
y_true = []

for p in paths:
    f = p.split('/')[6]
    y_true.append(label2id[f])

y_true = np.array(y_true)
y_true.shape

(422,)

In [6]:
from transformers import pipeline

pipe = pipeline("image-classification", "Nitish2801/beit-base-patch16-224-pt22k-ft22k-finetuned-footulcer")

2024-05-06 05:54:22.720798: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 05:54:22.720923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 05:54:22.864504: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/781 [00:00<?, ?B/s]

In [7]:
results = pipe(paths, num_workers=2, batch_size=8, top_k=1)
results[:5]

[[{'label': 'DM Group', 'score': 0.999954104423523}],
 [{'label': 'Control Group', 'score': 0.9551630616188049}],
 [{'label': 'DM Group', 'score': 0.9999659061431885}],
 [{'label': 'DM Group', 'score': 0.894271731376648}],
 [{'label': 'DM Group', 'score': 0.9999396800994873}]]

In [8]:
y_pred = []

for l in results:
    f = l[0]['label']
    y_pred.append(label2id[f])

y_pred = np.array(y_pred)
y_pred.shape

(422,)

In [9]:
label2id

{'Control Group': 0, 'DM Group': 1}

In [10]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=["Control Group", "DM Group"]))

               precision    recall  f1-score   support

Control Group       0.90      0.83      0.86       170
     DM Group       0.89      0.94      0.91       252

     accuracy                           0.89       422
    macro avg       0.89      0.88      0.89       422
 weighted avg       0.89      0.89      0.89       422

